# mplstyle and DRY for storytelling in matplotlib

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import duckdb

#om jag kör detta i Python script måste jag använda pathlib. 
#Path(__file__).parent / "styles.mplstyle"
#Men nu i notebook kan man gå raka vägen.
plt.style.use("styles/base.mplstyle")

df = pd.read_csv("data/jobs_in_data_2024.csv")
df.head()

OSError: 'styles/base.mplstyle' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

In [2]:
df.shape

(14199, 12)

In [ ]:
df_job_categories = df["job_category"].value_counts()

job_category
Data Science and Research         4675
Data Engineering                  3157
Data Analysis                     2204
Machine Learning and AI           2148
Leadership and Management          791
BI and Visualization               600
Data Architecture and Modeling     419
Data Management and Strategy       115
Data Quality and Operations         79
Cloud and Database                  11
Name: count, dtype: int64